データの前処理

1. ファイルを指定してデータを読み込む
2. タイムスタンプをdatetimeオブジェクトにする
3. 国を大州と地域に分割する　←　カラムを追加する
4. カラムをcategoryにする
5. コメントのある／なしフラグを追加する　←　カラムを追加する
6. （他にあれば追加する）

In [8]:
from pathlib import Path
import pandas as pd
import altair as alt
import titanite as ti

print(f"Pandas {pd.__version__}")
print(f"Altair {alt.__version__}")
print(f"Titanite {ti.__version__}")

Pandas 2.0.3
Altair 5.0.1
Titanite 0.1.0


設定ファイルを読み込む

In [9]:
fname = "../sandbox/config.toml"
config = ti.Config(fname=fname)
config.load()
# config.questions
# config.choices
# dir(config)

データを読み込む

In [10]:
fname = Path("../data/test_data/20230715_icrc2023_diversity_presurvey_answers.csv")
data = pd.read_csv(fname, skiprows=1)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          258 non-null    object 
 1   q1                 258 non-null    object 
 2   q2                 258 non-null    object 
 3   q3                 258 non-null    object 
 4   q4                 258 non-null    object 
 5   q5                 258 non-null    object 
 6   q6                 258 non-null    object 
 7   q7                 258 non-null    object 
 8   q8                 258 non-null    object 
 9   q9                 258 non-null    object 
 10  q10                258 non-null    float64
 11  q11                258 non-null    object 
 12  q12_genderbalance  258 non-null    object 
 13  q12_diversity      258 non-null    object 
 14  q12_equity         258 non-null    object 
 15  q12_inclusion      258 non-null    object 
 16  q13                258 non

日付のカラムを``datetime``オブジェクトに変換する

In [12]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

カラムの値を置換して整理する
- "regional" が "Prefer not to answer" の場合、"subregional" も "Prefer not to answer"
- "regional" が "Oceania" の場合、"subregional"も"Oceania"
- "No Interest" を "No interest"に修正する

In [15]:
data["q3"] = data["q3"].replace(
    {"Prefer not to answer": "Prefer not to answer / Prefer not to answer",
    "Oceania": "Oceania / Oceania"}
)
data["q4"] = data["q4"].replace(
    {"Prefer not to answer": "Prefer not to answer / Prefer not to answer",
    "Oceania": "Oceania / Oceania"}
)
data["q14"] = data["q14"].replace(
    {"No Interest": "No interest"}
)

In [16]:
_q3 = data["q3"].str.split("/", expand=True)
_q3[0] = _q3[0].str.strip()
_q3[1] = _q3[1].str.strip()
_q3 = _q3.rename(columns={0: "q3_regional", 1: "q3_subregional"})

_q4 = data["q4"].str.split("/", expand=True)
_q4[0] = _q4[0].str.strip()
_q4[1] = _q4[1].str.strip()
_q4 = _q4.rename(columns={0: "q4_regional", 1: "q4_subregional"})

In [17]:
data = pd.concat([data, _q3], axis=1)
data = pd.concat([data, _q4], axis=1)
#data

In [18]:
data.columns

Index(['timestamp', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9',
       'q10', 'q11', 'q12_genderbalance', 'q12_diversity', 'q12_equity',
       'q12_inclusion', 'q13', 'q14', 'q15', 'q16', 'q17_genderbalance',
       'q17_diversity', 'q17_equity', 'q17_inclusion', 'q18', 'q19', 'q20',
       'q21', 'q22', 'q3_regional', 'q3_subregional', 'q4_regional',
       'q4_subregional'],
      dtype='object')

カテゴリー型に変換する

In [19]:
category = config.categories()
data["q1"] = data["q1"].astype(category["age"])
data["q2"] = data["q2"].astype(category["gender"])
data["q3"] = data["q3"].astype(category["geoscheme"])
data["q3_regional"] = data["q3_regional"].astype(category["regional"])
data["q3_subregional"] = data["q3_subregional"].astype(category["subregional"])
data["q4"] = data["q4"].astype(category["geoscheme"])
data["q4_regional"] = data["q4_regional"].astype(category["regional"])
data["q4_subregional"] = data["q4_subregional"].astype(category["subregional"])
data["q5"] = data["q5"].astype(category["job_title"])
data["q6"] = data["q6"].astype(category["research_group"])
data["q7"] = data["q7"].astype(category["research_field"])
data["q8"] = data["q8"].astype(category["research_years"])
data["q9"] = data["q9"].astype(category["yes_no"])
# data["q10"]
data["q11"] = data["q11"].astype(category["yes_no"])
data["q12_genderbalance"] = data["q12_genderbalance"].astype(category["good_poor"])
data["q12_diversity"] = data["q12_diversity"].astype(category["good_poor"])
data["q12_equity"] = data["q12_equity"].astype(category["good_poor"])
data["q12_inclusion"] = data["q12_inclusion"].astype(category["good_poor"])
# data["q13"]
data["q14"] = data["q14"].astype(category["good_poor"])
# data["q15"]
# data["q16"]
data["q17_genderbalance"] = data["q17_genderbalance"].astype(category["agree_disagree"])
data["q17_diversity"] = data["q17_diversity"].astype(category["agree_disagree"])
data["q17_equity"] = data["q17_equity"].astype(category["agree_disagree"])
data["q17_inclusion"] = data["q17_inclusion"].astype(category["agree_disagree"])
# data["q18"]
data["q19"] = data["q19"].astype(category["school"])
# data["q20"]
# data["q21"]
# data["q22"]

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          258 non-null    datetime64[ns]
 1   q1                 258 non-null    category      
 2   q2                 258 non-null    category      
 3   q3                 249 non-null    category      
 4   q4                 247 non-null    category      
 5   q5                 258 non-null    category      
 6   q6                 258 non-null    category      
 7   q7                 235 non-null    category      
 8   q8                 258 non-null    category      
 9   q9                 258 non-null    category      
 10  q10                258 non-null    float64       
 11  q11                258 non-null    category      
 12  q12_genderbalance  258 non-null    category      
 13  q12_diversity      258 non-null    category      
 14  q12_equity

In [21]:
data.to_csv("tmp_main_data.csv", index=False)